# Current Model

In [1]:
#imports
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization, BatchNormalization
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from twilio.rest import Client
import csv
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
import joblib

In [10]:
def preprocess_data(csv_file):
    data = pd.read_csv(csv_file)

    X = data.drop('c0', axis=1)
    Y = data['c0'].values

    categorical_cols = ['Kmer']
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    X_categorical_encoded = encoder.fit_transform(X[categorical_cols])

    numerical_cols = ['skew', 'ratio']
    scaler_X = StandardScaler()
    X_numerical_scaled = scaler_X.fit_transform(X[numerical_cols])
    
    joblib.dump(encoder, 'encoder.pkl')
    joblib.dump(scaler_X, 'standard_scaler.pkl')

    return X_categorical_encoded, X_numerical_scaled, Y

def create_model(input_shapes):
    input_cat = tf.keras.layers.Input(shape=(input_shapes[0],))
    cat_branch = tf.keras.layers.Dense(128, activation='tanh')(input_cat)
    cat_branch = tf.keras.layers.Dropout(0.2)(cat_branch)

    input_num = tf.keras.layers.Input(shape=(input_shapes[1],))
    num_branch = tf.keras.layers.Dense(128, activation='relu')(input_num)
    num_branch = tf.keras.layers.Dropout(0.2)(num_branch)

    merged = tf.keras.layers.concatenate([cat_branch, num_branch])

    merged = tf.keras.layers.Dense(64, activation='relu')(merged)
    merged = tf.keras.layers.Dropout(0.2)(merged)
    merged = tf.keras.layers.Dense(32, activation='sigmoid')(merged)
    merged = tf.keras.layers.Dropout(0.2)(merged)
    output = tf.keras.layers.Dense(1, activation='linear')(merged)

    model = tf.keras.Model(inputs=[input_cat, input_num], outputs=output)
    return model

def compile_model(model):
    model.compile(optimizer='adam',loss='mse', metrics = ['mae', 'mse'])
    return model

def get_model_layers_info(model):
    model_summary = []
    for layer in model.layers:
        layer_info = {
            "Layer_Name": layer.name,
            "Layer_Type": type(layer).__name__,
            "Number_of_Nodes": layer.units if hasattr(layer, 'units') else None,
            "Activation_Function": layer.activation.__name__ if hasattr(layer, 'activation') else None
        }
        model_summary.append(layer_info)
    return model_summary
    
def write_model_summary_to_csv(model, model_summary, output_file, X_cat_test, X_num_test, Y_test, data_set):
    predictions = model.predict([X_cat_test, X_num_test])
    true_targets = Y_test
    predictions = np.squeeze(predictions)
    true_targets = np.squeeze(true_targets)
    mae = mean_absolute_error(true_targets, predictions)
    mse = mean_squared_error(true_targets, predictions)
    r2 = r2_score(true_targets, predictions)
    print(f"MAE: {mae}, MSE: {mse}, r2: {r2}, data set: {data_set}")
    
    with open(output_file, mode='a', newline='') as file:
        fieldnames = ["Layer_Name", "Layer_Type", "Number_of_Nodes", "Activation_Function"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writerow({"Layer_Name": "MAE", "Layer_Type": str(mae), "Number_of_Nodes": "MSE", "Activation_Function": str(mse)})
        writer.writerow({"Layer_Name": "R^2", "Layer_Type": str(r2), "Number_of_Nodes": "Data:", "Activation_Function": data_set})
        writer.writeheader()
        for layer_info in model_summary:
            writer.writerow(layer_info)

# Load and preprocess data
data_set = 'chrV.csv'
X_cat, X_num, Y = preprocess_data(data_set)
X_cat_train, X_cat_test, X_num_train, X_num_test, Y_train, Y_test = train_test_split(X_cat, X_num, Y, test_size=0.05, random_state=42)


early_stopping = EarlyStopping(monitor='mse', patience=10, restore_best_weights=True)
model = create_model(input_shapes=(X_cat_train.shape[1], X_num_train.shape[1]))
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
model.fit([X_cat_train, X_num_train], Y_train, epochs=20, batch_size=64, callbacks=[early_stopping])

model_summary = get_model_layers_info(model)
output_file = "1_model_summary.csv"
write_model_summary_to_csv(model, model_summary, output_file, X_cat_test, X_num_test, Y_test, data_set)


model.save('kmer_model.h5')


C:\Users\chris\anaconda3\envs\DncycP\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/20
162/162 [==============================] - 1s 4ms/step - loss: 0.2214 - mae: 0.4253 - mse: 0.2214
Epoch 2/20
162/162 [==============================] - 1s 4ms/step - loss: 0.1807 - mae: 0.4190 - mse: 0.1807
Epoch 3/20
162/162 [==============================] - 1s 4ms/step - loss: 0.1791 - mae: 0.4192 - mse: 0.1791
Epoch 4/20
162/162 [==============================] - 1s 4ms/step - loss: 0.1770 - mae: 0.4172 - mse: 0.1770
Epoch 5/20
162/162 [==============================] - 1s 4ms/step - loss: 0.1625 - mae: 0.3905 - mse: 0.1625
Epoch 6/20
162/162 [==============================] - 1s 4ms/step - loss: 0.0876 - mae: 0.2492 - mse: 0.0876
Epoch 7/20
162/162 [==============================] - 1s 4ms/step - loss: 0.0587 - mae: 0.1935 - mse: 0.0587
Epoch 8/20
162/162 [==============================] - 1s 4ms/step - loss: 0.0430 - mae: 0.1604 - mse: 0.0430
Epoch 9/20
162/162 [==============================] - 1s 4ms/step - loss: 0.0350 - mae: 0.1420 - mse: 0.0350
Epoch 10/20
162/162

## used for retraining

In [ ]:

loaded_model = tf.keras.models.load_model('kmer_model.h5')

# Load and preprocess the new dataset
new_data_set = 'tiling.csv'
X_cat, X_num, Y = preprocess_data(new_data_set)  # Make sure to define your preprocess_data function

# Split new data into training and testing sets
_cat_train, X_cat_test, X_num_train, X_num_test, Y_train, Y_test = train_test_split(X_cat, X_num, Y, test_size=0.05, random_state=42)

# Define early stopping
early_stopping = EarlyStopping(monitor='mse', patience=10, restore_best_weights=True)

# Continue training the loaded model with the new dataset
loaded_model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
loaded_model.fit([X_cat_train, X_num_train], Y_train, epochs=20, batch_size=64, callbacks=[early_stopping])

model_summary = get_model_layers_info(loaded_model)
output_file = "1_model_summary.csv"
write_model_summary_to_csv(loaded_model, model_summary, output_file, X_cat_test, X_num_test, Y_test, data_set)


loaded_model.save('kmer_model.h5')